In [2]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [20]:
!python ../scripts/mimiccxr/export_cxrlt2024_training_labels_for_fact_classification.py \
--fact_encoder_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_encoder_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)"

2024-07-18 21:05:58,186 - INFO - Loading CXR-LT 2024 training data
2024-07-18 21:05:59,546 - INFO - train_labels.shape: (258871, 40)
2024-07-18 21:05:59,546 - INFO - Computing class embeddings
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
  0%|                                                    | 0/40 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=441,505879158912416539).pkl
len(self.cache["hashes"]) = 599219
self.cache["embeddings"].shape = (599219, 128)
100%|███████████████████████████████████████████| 40/40 [00:00<00:00, 88.74it/s]
2024-07-18 21:06:00,221 - INFO - class_embeddings.shape: (40, 128)
2024-07-18 21:06:00,222 - INFO - Splitting the training data into train and validation sets
2024-07-18 21:06:01,152 - INFO - len(train_indices): 252608
2024-07-18 21:06:01,152 - INFO - len(val_indices): 6263
20

In [21]:
from medvqa.utils.files import load_pickle

tmp = load_pickle('/mnt/workspace/pamessina/medvqa-workspace/cache/mimiccxr/cxrlt2024_offical_training_labels_for_fact_classification.pkl')

In [22]:
tmp.keys()

dict_keys(['dicom_ids', 'labels', 'train_indices', 'val_indices', 'class_names', 'class_embeddings'])

In [23]:
tmp['class_embeddings'].shape

(40, 128)

In [38]:
# --use_mimiccxr_facts_for_train \
# --dicom_id_to_pos_neg_facts_filepath \
# "/mnt/workspace/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_facts=596394)(hash=469,647223496013944140).pkl" \
# --mimiccxr_exclude_noisy_images \
# --mimiccxr_balance_long_middle_short_tail \
# --mimiccxr_report_fact_nli_integrated_data_filepath \
# "/mnt/workspace/pamessina/medvqa-workspace/cache/mimiccxr_integrated_data(hash=1649,2670402002373726730).pkl" \
# --binary_multilabel_classif_loss_name "focal+bce+wbce" \
# --use_weighted_phrase_classifier_loss \
# --cluster_and_label_weights_for_facts_filepath \
# "/mnt/workspace/pamessina/medvqa-workspace/cache/mimiccxr_clusters_and_cluster_weights_for_facts(hash=226,607533638975659953).pkl" \
# --use_chest_imagenome_for_train \
# --chest_imagenome_bbox_phrase_embeddings_filepath \
# "/mnt/workspace/pamessina/medvqa-workspace/cache/chest_imagenome/bbox_phrase_embeddings(hash=333,1643894111267724539).pkl" \
# --use_vinbig_for_train \
# --vinbig_phrase_embeddings_filepath \
# "/mnt/workspace/pamessina/medvqa-workspace/cache/vinbig/label_phrase_embeddings(hash=325,2238524744738711392).pkl" \
# --vinbig_training_data_mode "all" \
# --use_chexlocalize_for_train \
# --chexlocalize_class_phrase_embeddings_filepath \
# "/mnt/workspace/pamessina/medvqa-workspace/cache/chexlocalize/class_phrase_embeddings(hash=528,2353352898845085012).pkl" \
# --use_chexpert_for_train \
# --chexpert_class_phrase_embeddings_filepath \
# "/mnt/workspace/pamessina/medvqa-workspace/cache/chexpert/class_phrase_embeddings(hash=592,2793798096738783992).pkl" \
# --use_iuxray_for_train \
# --iuxray_image_id_to_pos_neg_facts_filepath \
# "/mnt/workspace/pamessina/medvqa-workspace/cache/iuxray/image_id_to_pos_neg_facts(hash=450,1832071235913899237).pkl" \

# --use_cxrlt2024_custom_labels \
# --cxrlt2024_custom_dicom_id_to_pos_neg_facts_filepath \
# "/mnt/workspace/pamessina/medvqa-workspace/cache/mimiccxr/cxrlt2024_integrated_nli_queries_for_fact_classification.pkl" \

# --cxrlt2024_weight 4.0 \
# --mimiccxr_facts_weight 1.5 \
# --chest_imagenome_anatlocs_weight 1.0 \
# --iuxray_weight 1.0 \
# --vinbig_weight 1.5 \
# --chexpert_weight 0.4 \
# --chexlocalize_weight 0.4 \


!python ../train_phrase_grounding.py \
--pretrained_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/phrase_grounding/20240716_075455_mim-facts+chst-img-anat+cxrlt2024+vinbig+chexloc+chxp+iuxray_PhraseGrounder(dn121,FiLM_SigmoidAttention,128,256,256,256,1,256-128)" \
--epochs 200 \
--batches_per_epoch 500 \
--max_images_per_batch 50 \
--max_phrases_per_batch 800 \
--max_phrases_per_image 50 \
--val_batch_size_factor 2.0 \
--raw_image_encoding "densenet-121" \
--image_size 416 416 \
--image_local_feat_size 1024 \
--num_regions 169 \
--regions_width 13 \
--regions_height 13 \
--phrase_embedding_size 128 \
--phrase_grounding_mode "film_layers_plus_sigmoid_attention_and_custom_classifier" \
--visual_feature_proj_size 256 \
--visual_grounding_hidden_size 256 \
--phrase_mlp_hidden_dims 256 128 \
--num_train_workers 2 \
--num_val_workers 2 \
--gradient_accumulation_steps 10 \
--optimizer_name "adamw" \
--scheduler "exp-warmup+decay+cyclicdecay" \
--lr 1e-6 \
--warmup_decay_and_cyclic_decay_args "1e-6,3,8e-5,8,1e-6,8e-5,8,1e-6" \
--mimiccxr_exclude_noisy_images \
--binary_multilabel_classif_loss_name "focal+bce+wbce" \
--use_attention_regularization_loss \
--attention_supervision_loss_weight 2.0 \
--phrase_classifier_loss_weight 2.0 \
--foreground_loss_weight 1.5 \
--mask_exponent 0.4 \
--use_cxrlt2024_challenge_split \
--use_cxrlt2024_official_labels \
--cxrlt2024_official_training_labels_for_fact_classification_filepath \
"/mnt/workspace/pamessina/medvqa-workspace/cache/mimiccxr/cxrlt2024_offical_training_labels_for_fact_classification.pkl" \
--save

script's arguments:
   epochs: 200
   batches_per_epoch: 500
   max_images_per_batch: 50
   max_phrases_per_batch: 800
   max_phrases_per_image: 50
   val_batch_size_factor: 2.0
   checkpoint_folder: None
   pretrained_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/phrase_grounding/20240716_075455_mim-facts+chst-img-anat+cxrlt2024+vinbig+chexloc+chxp+iuxray_PhraseGrounder(dn121,FiLM_SigmoidAttention,128,256,256,256,1,256-128)
   pretrained_checkpoint_folder_paths: None
   freeze_image_encoder: False
   raw_image_encoding: densenet-121
   num_regions: 169
   image_local_feat_size: 1024
   image_encoder_pretrained_weights_path: None
   yolov8_model_name_or_path: None
   yolov8_model_alias: None
   phrase_embedding_size: 128
   regions_width: 13
   regions_height: 13
   qkv_size: None
   phrase_grounding_mode: film_layers_plus_sigmoid_attention_and_custom_classifier
   phrase_classifier_hidden_size: None
   transf_d_model: None
   transf_nhead: None
   tran

Group sizes: [33793, 24259, 23634, 22006, 18662, 14197, 13915, 10283, 9205, 8905, 8212, 7907, 6861, 4835, 3680, 3633, 3399, 3276, 2934, 2775, 2733, 2638, 2548, 2420, 2051, 1956, 1875, 1643, 1380, 1087, 836, 749, 685, 631, 608, 580, 499, 194, 164, 158, 120]
  len(indices) = 33793, weight = 3405.0837942874055
  len(indices) = 24259, weight = 3090.5922382876397
  len(indices) = 23634, weight = 3066.6849850295202
  len(indices) = 22006, weight = 3001.9433897188514
  len(indices) = 18662, weight = 2855.9239691131966
  len(indices) = 14197, weight = 2624.2451479240017
  len(indices) = 13915, weight = 2607.758881815128
  len(indices) = 10283, weight = 2367.5129962345654
  len(indices) = 9205, weight = 2283.3865606003296
  len(indices) = 8905, weight = 2258.6098418627666
  len(indices) = 8212, weight = 2198.7840641351168
  len(indices) = 7907, weight = 2171.20140287369
  len(indices) = 6861, weight = 2069.8461094055197
  len(indices) = 4835, weight = 1833.452903391974
  len(indices) = 3680, we

New checkpoint saved: checkpoint_9_cxss+cxss+cxuc=0.5179.pt
---- Epoch 10/200
(1) Training stage (lr = 0.000003) ...
loss 2.47046, cxrlt2024o_att_reg_loss 0.20322, cxrlt2024o_phrcls_loss 2.26724, cxrlt2024o_prc_auc 0.21781, 102.89 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.11610, cxrlt2024o_phrcls_loss 1.10636, cxrlt2024o_prc_auc 0.20473, 34.74 secs
---- Epoch 11/200
(1) Training stage (lr = 0.000002) ...
loss 2.47018, cxrlt2024o_att_reg_loss 0.20356, cxrlt2024o_phrcls_loss 2.26661, cxrlt2024o_prc_auc 0.21656, 103.22 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.11561, cxrlt2024o_phrcls_loss 1.10229, cxrlt2024o_prc_auc 0.20481, 34.51 secs
New checkpoint saved: checkpoint_11_cxss+cxss+cxuc=0.5182.pt
---- Epoch 12/200
(1) Training stage (lr = 0.000001) ...
loss 2.47489, cxrlt2024o_att_reg_loss 0.20322, cxrlt2024o_phrcls_loss 2.27167, cxrlt2024o_prc_auc 0.21667, 102.77 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.11440, cxrlt2024o_phrcls_loss 1.10583, cxr

New checkpoint saved: checkpoint_32_cxss+cxss+cxuc=0.5322.pt
---- Epoch 33/200
(1) Training stage (lr = 0.000007) ...
loss 2.31730, cxrlt2024o_att_reg_loss 0.19007, cxrlt2024o_phrcls_loss 2.12723, cxrlt2024o_prc_auc 0.31772, 101.80 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09988, cxrlt2024o_phrcls_loss 1.08164, cxrlt2024o_prc_auc 0.22768, 35.12 secs
New checkpoint saved: checkpoint_33_cxss+cxss+cxuc=0.5344.pt
---- Epoch 34/200
(1) Training stage (lr = 0.000003) ...
loss 2.29478, cxrlt2024o_att_reg_loss 0.19025, cxrlt2024o_phrcls_loss 2.10454, cxrlt2024o_prc_auc 0.33305, 101.67 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.10039, cxrlt2024o_phrcls_loss 1.08265, cxrlt2024o_prc_auc 0.22477, 34.85 secs
---- Epoch 35/200
(1) Training stage (lr = 0.000002) ...
loss 2.29839, cxrlt2024o_att_reg_loss 0.18940, cxrlt2024o_phrcls_loss 2.10898, cxrlt2024o_prc_auc 0.33777, 102.99 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.10150, cxrlt2024o_phrcls_loss 1.07869, cx

cxrlt2024o_att_reg_loss 0.09724, cxrlt2024o_phrcls_loss 1.07779, cxrlt2024o_prc_auc 0.22598, 35.56 secs
New checkpoint saved: checkpoint_57_cxss+cxss+cxuc=0.5382.pt
---- Epoch 58/200
(1) Training stage (lr = 0.000003) ...
loss 2.14936, cxrlt2024o_att_reg_loss 0.18500, cxrlt2024o_phrcls_loss 1.96436, cxrlt2024o_prc_auc 0.42561, 103.05 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09763, cxrlt2024o_phrcls_loss 1.09551, cxrlt2024o_prc_auc 0.22363, 35.76 secs
---- Epoch 59/200
(1) Training stage (lr = 0.000002) ...
loss 2.14685, cxrlt2024o_att_reg_loss 0.18415, cxrlt2024o_phrcls_loss 1.96270, cxrlt2024o_prc_auc 0.42559, 102.86 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09780, cxrlt2024o_phrcls_loss 1.08298, cxrlt2024o_prc_auc 0.22646, 35.88 secs
New checkpoint saved: checkpoint_59_cxss+cxss+cxuc=0.5388.pt
---- Epoch 60/200
(1) Training stage (lr = 0.000001) ...
loss 2.17158, cxrlt2024o_att_reg_loss 0.18358, cxrlt2024o_phrcls_loss 1.98800, cxrlt2024o_prc_auc 0.41997, 

cxrlt2024o_att_reg_loss 0.09654, cxrlt2024o_phrcls_loss 1.10932, cxrlt2024o_prc_auc 0.22662, 36.46 secs
New checkpoint saved: checkpoint_82_cxss+cxss+cxuc=0.5403.pt
---- Epoch 83/200
(1) Training stage (lr = 0.000002) ...
loss 2.02716, cxrlt2024o_att_reg_loss 0.18102, cxrlt2024o_phrcls_loss 1.84614, cxrlt2024o_prc_auc 0.49848, 103.31 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09818, cxrlt2024o_phrcls_loss 1.11127, cxrlt2024o_prc_auc 0.22677, 36.64 secs
---- Epoch 84/200
(1) Training stage (lr = 0.000001) ...
loss 2.04352, cxrlt2024o_att_reg_loss 0.18205, cxrlt2024o_phrcls_loss 1.86148, cxrlt2024o_prc_auc 0.49396, 103.11 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09687, cxrlt2024o_phrcls_loss 1.10813, cxrlt2024o_prc_auc 0.22788, 36.19 secs
New checkpoint saved: checkpoint_84_cxss+cxss+cxuc=0.5405.pt
---- Epoch 85/200
(1) Training stage (lr = 0.000080) ...
loss 2.08077, cxrlt2024o_att_reg_loss 0.18491, cxrlt2024o_phrcls_loss 1.89586, cxrlt2024o_prc_auc 0.46120, 

loss 1.97598, cxrlt2024o_att_reg_loss 0.18180, cxrlt2024o_phrcls_loss 1.79418, cxrlt2024o_prc_auc 0.52032, 102.62 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09192, cxrlt2024o_phrcls_loss 1.13856, cxrlt2024o_prc_auc 0.21800, 37.84 secs
---- Epoch 110/200
(1) Training stage (lr = 0.000043) ...
loss 1.95464, cxrlt2024o_att_reg_loss 0.18075, cxrlt2024o_phrcls_loss 1.77388, cxrlt2024o_prc_auc 0.52314, 101.95 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09780, cxrlt2024o_phrcls_loss 1.16225, cxrlt2024o_prc_auc 0.22756, 37.20 secs
---- Epoch 111/200
(1) Training stage (lr = 0.000023) ...
loss 1.93309, cxrlt2024o_att_reg_loss 0.17980, cxrlt2024o_phrcls_loss 1.75329, cxrlt2024o_prc_auc 0.54131, 102.70 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09620, cxrlt2024o_phrcls_loss 1.17581, cxrlt2024o_prc_auc 0.22768, 37.26 secs
---- Epoch 112/200
(1) Training stage (lr = 0.000012) ...
loss 1.92234, cxrlt2024o_att_reg_loss 0.18016, cxrlt2024o_phrcls_loss 1.74218, cxr

---- Epoch 135/200
(1) Training stage (lr = 0.000023) ...
loss 1.84449, cxrlt2024o_att_reg_loss 0.17946, cxrlt2024o_phrcls_loss 1.66502, cxrlt2024o_prc_auc 0.58545, 100.38 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09610, cxrlt2024o_phrcls_loss 1.19698, cxrlt2024o_prc_auc 0.22366, 50.84 secs
---- Epoch 136/200
(1) Training stage (lr = 0.000012) ...
loss 1.80297, cxrlt2024o_att_reg_loss 0.17663, cxrlt2024o_phrcls_loss 1.62634, cxrlt2024o_prc_auc 0.59930, 104.41 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09561, cxrlt2024o_phrcls_loss 1.20828, cxrlt2024o_prc_auc 0.22278, 52.77 secs
---- Epoch 137/200
(1) Training stage (lr = 0.000007) ...
loss 1.80014, cxrlt2024o_att_reg_loss 0.17865, cxrlt2024o_phrcls_loss 1.62149, cxrlt2024o_prc_auc 0.60594, 102.66 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09633, cxrlt2024o_phrcls_loss 1.20801, cxrlt2024o_prc_auc 0.22610, 48.72 secs
---- Epoch 138/200
(1) Training stage (lr = 0.000003) ...
loss 1.79161, cxrlt2024o

In [41]:
!python ../train_phrase_grounding.py \
--pretrained_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/phrase_grounding/20240719_101753_cxrlt2024(Off)_PhraseGrounder(dn121,FiLM_SigmoidAttention,128,256,256,256,1,256-128)" \
--epochs 200 \
--batches_per_epoch 500 \
--max_images_per_batch 50 \
--max_phrases_per_batch 800 \
--max_phrases_per_image 50 \
--val_batch_size_factor 2.0 \
--raw_image_encoding "densenet-121" \
--img_aug_mode "random-color-and-spatial" \
--image_encoder_dropout_p 0.1 \
--image_size 416 416 \
--image_local_feat_size 1024 \
--num_regions 169 \
--regions_width 13 \
--regions_height 13 \
--phrase_embedding_size 128 \
--phrase_grounding_mode "film_layers_plus_sigmoid_attention_and_custom_classifier" \
--visual_feature_proj_size 256 \
--visual_grounding_hidden_size 256 \
--phrase_mlp_hidden_dims 256 128 \
--num_train_workers 2 \
--num_val_workers 2 \
--gradient_accumulation_steps 10 \
--optimizer_name "adamw" \
--scheduler "exp-warmup+decay+cyclicdecay" \
--lr 1e-6 \
--warmup_decay_and_cyclic_decay_args "1e-6,3,8e-5,8,1e-6,8e-5,8,1e-6" \
--mimiccxr_exclude_noisy_images \
--binary_multilabel_classif_loss_name "focal+bce+wbce" \
--use_attention_regularization_loss \
--attention_supervision_loss_weight 2.0 \
--phrase_classifier_loss_weight 2.0 \
--foreground_loss_weight 1.5 \
--mask_exponent 0.4 \
--use_cxrlt2024_challenge_split \
--use_cxrlt2024_official_labels \
--cxrlt2024_official_training_labels_for_fact_classification_filepath \
"/mnt/workspace/pamessina/medvqa-workspace/cache/mimiccxr/cxrlt2024_offical_training_labels_for_fact_classification.pkl" \
--save

script's arguments:
   epochs: 200
   batches_per_epoch: 500
   max_images_per_batch: 50
   max_phrases_per_batch: 800
   max_phrases_per_image: 50
   val_batch_size_factor: 2.0
   checkpoint_folder: None
   pretrained_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/phrase_grounding/20240719_101753_cxrlt2024(Off)_PhraseGrounder(dn121,FiLM_SigmoidAttention,128,256,256,256,1,256-128)
   pretrained_checkpoint_folder_paths: None
   freeze_image_encoder: False
   raw_image_encoding: densenet-121
   num_regions: 169
   image_local_feat_size: 1024
   image_encoder_pretrained_weights_path: None
   image_encoder_dropout_p: 0.1
   yolov8_model_name_or_path: None
   yolov8_model_alias: None
   phrase_embedding_size: 128
   regions_width: 13
   regions_height: 13
   qkv_size: None
   phrase_grounding_mode: film_layers_plus_sigmoid_attention_and_custom_classifier
   phrase_classifier_hidden_size: None
   transf_d_model: None
   transf_nhead: None
   transf_dim_feedfor

Group sizes: [33793, 24259, 23634, 22006, 18662, 14197, 13915, 10283, 9205, 8905, 8212, 7907, 6861, 4835, 3680, 3633, 3399, 3276, 2934, 2775, 2733, 2638, 2548, 2420, 2051, 1956, 1875, 1643, 1380, 1087, 836, 749, 685, 631, 608, 580, 499, 194, 164, 158, 120]
  len(indices) = 33793, weight = 3405.0837942874055
  len(indices) = 24259, weight = 3090.5922382876397
  len(indices) = 23634, weight = 3066.6849850295202
  len(indices) = 22006, weight = 3001.9433897188514
  len(indices) = 18662, weight = 2855.9239691131966
  len(indices) = 14197, weight = 2624.2451479240017
  len(indices) = 13915, weight = 2607.758881815128
  len(indices) = 10283, weight = 2367.5129962345654
  len(indices) = 9205, weight = 2283.3865606003296
  len(indices) = 8905, weight = 2258.6098418627666
  len(indices) = 8212, weight = 2198.7840641351168
  len(indices) = 7907, weight = 2171.20140287369
  len(indices) = 6861, weight = 2069.8461094055197
  len(indices) = 4835, weight = 1833.452903391974
  len(indices) = 3680, we

cxrlt2024o_att_reg_loss 0.09495, cxrlt2024o_phrcls_loss 1.09655, cxrlt2024o_prc_auc 0.22261, 36.67 secs
New checkpoint saved: checkpoint_10_cxss+cxss+cxuc=0.4553.pt
---- Epoch 11/200
(1) Training stage (lr = 0.000002) ...
loss 2.37101, cxrlt2024o_att_reg_loss 0.18732, cxrlt2024o_phrcls_loss 2.18369, cxrlt2024o_prc_auc 0.27027, 104.37 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09604, cxrlt2024o_phrcls_loss 1.09439, cxrlt2024o_prc_auc 0.22183, 37.03 secs
---- Epoch 12/200
(1) Training stage (lr = 0.000001) ...
loss 2.35170, cxrlt2024o_att_reg_loss 0.18702, cxrlt2024o_phrcls_loss 2.16467, cxrlt2024o_prc_auc 0.27159, 104.48 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09612, cxrlt2024o_phrcls_loss 1.09209, cxrlt2024o_prc_auc 0.22241, 36.34 secs
New checkpoint saved: checkpoint_12_cxss+cxss+cxuc=0.4554.pt
---- Epoch 13/200
(1) Training stage (lr = 0.000080) ...
loss 2.38713, cxrlt2024o_att_reg_loss 0.18872, cxrlt2024o_phrcls_loss 2.19841, cxrlt2024o_prc_auc 0.26008, 

(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09303, cxrlt2024o_phrcls_loss 1.07791, cxrlt2024o_prc_auc 0.22879, 37.56 secs
---- Epoch 35/200
(1) Training stage (lr = 0.000002) ...
loss 2.27319, cxrlt2024o_att_reg_loss 0.18384, cxrlt2024o_phrcls_loss 2.08935, cxrlt2024o_prc_auc 0.32961, 103.12 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09361, cxrlt2024o_phrcls_loss 1.08049, cxrlt2024o_prc_auc 0.22947, 36.82 secs
---- Epoch 36/200
(1) Training stage (lr = 0.000001) ...
loss 2.29361, cxrlt2024o_att_reg_loss 0.18417, cxrlt2024o_phrcls_loss 2.10945, cxrlt2024o_prc_auc 0.31562, 102.86 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09332, cxrlt2024o_phrcls_loss 1.07835, cxrlt2024o_prc_auc 0.23021, 39.95 secs
---- Epoch 37/200
(1) Training stage (lr = 0.000080) ...
loss 2.30312, cxrlt2024o_att_reg_loss 0.18545, cxrlt2024o_phrcls_loss 2.11768, cxrlt2024o_prc_auc 0.30349, 103.69 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09614, cxrlt2024o_phrcls_loss 1.10

cxrlt2024o_att_reg_loss 0.09267, cxrlt2024o_phrcls_loss 1.06603, cxrlt2024o_prc_auc 0.23341, 36.42 secs
New checkpoint saved: checkpoint_60_cxss+cxss+cxuc=0.4666.pt
---- Epoch 61/200
(1) Training stage (lr = 0.000080) ...
loss 2.27520, cxrlt2024o_att_reg_loss 0.18401, cxrlt2024o_phrcls_loss 2.09118, cxrlt2024o_prc_auc 0.32333, 103.54 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09576, cxrlt2024o_phrcls_loss 1.09551, cxrlt2024o_prc_auc 0.23057, 36.79 secs
---- Epoch 62/200
(1) Training stage (lr = 0.000043) ...
loss 2.27472, cxrlt2024o_att_reg_loss 0.18475, cxrlt2024o_phrcls_loss 2.08997, cxrlt2024o_prc_auc 0.31980, 104.31 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09688, cxrlt2024o_phrcls_loss 1.09075, cxrlt2024o_prc_auc 0.23305, 36.82 secs
---- Epoch 63/200
(1) Training stage (lr = 0.000023) ...
loss 2.25659, cxrlt2024o_att_reg_loss 0.18340, cxrlt2024o_phrcls_loss 2.07320, cxrlt2024o_prc_auc 0.33278, 104.33 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 

In [43]:
!python ../train_phrase_grounding.py \
--pretrained_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/phrase_grounding/20240719_164036_cxrlt2024(Off)_PhraseGrounder(dn121,FiLM_SigmoidAttention,128,256,256,256,1,256-128)" \
--epochs 200 \
--batches_per_epoch 500 \
--max_images_per_batch 50 \
--max_phrases_per_batch 800 \
--max_phrases_per_image 50 \
--val_batch_size_factor 2.0 \
--raw_image_encoding "densenet-121" \
--img_aug_mode "random-color-and-spatial" \
--image_size 416 416 \
--image_local_feat_size 1024 \
--num_regions 169 \
--regions_width 13 \
--regions_height 13 \
--phrase_embedding_size 128 \
--phrase_grounding_mode "film_layers_plus_sigmoid_attention_and_custom_classifier" \
--visual_feature_proj_size 256 \
--visual_grounding_hidden_size 256 \
--phrase_mlp_hidden_dims 256 128 \
--num_train_workers 2 \
--num_val_workers 2 \
--gradient_accumulation_steps 10 \
--optimizer_name "adamw" \
--scheduler "exp-warmup+decay+cyclicdecay" \
--lr 1e-6 \
--warmup_decay_and_cyclic_decay_args "1e-6,3,8e-5,8,1e-6,8e-5,8,1e-6" \
--mimiccxr_exclude_noisy_images \
--binary_multilabel_classif_loss_name "focal+bce+wbce" \
--use_attention_regularization_loss \
--attention_supervision_loss_weight 2.0 \
--phrase_classifier_loss_weight 2.0 \
--foreground_loss_weight 1.5 \
--mask_exponent 0.4 \
--use_cxrlt2024_challenge_split \
--use_cxrlt2024_official_labels \
--cxrlt2024_official_training_labels_for_fact_classification_filepath \
"/mnt/workspace/pamessina/medvqa-workspace/cache/mimiccxr/cxrlt2024_offical_training_labels_for_fact_classification.pkl" \
--save

script's arguments:
   epochs: 200
   batches_per_epoch: 500
   max_images_per_batch: 50
   max_phrases_per_batch: 800
   max_phrases_per_image: 50
   val_batch_size_factor: 2.0
   checkpoint_folder: None
   pretrained_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/phrase_grounding/20240719_164036_cxrlt2024(Off)_PhraseGrounder(dn121,FiLM_SigmoidAttention,128,256,256,256,1,256-128)
   pretrained_checkpoint_folder_paths: None
   freeze_image_encoder: False
   raw_image_encoding: densenet-121
   num_regions: 169
   image_local_feat_size: 1024
   image_encoder_pretrained_weights_path: None
   image_encoder_dropout_p: 0
   yolov8_model_name_or_path: None
   yolov8_model_alias: None
   phrase_embedding_size: 128
   regions_width: 13
   regions_height: 13
   qkv_size: None
   phrase_grounding_mode: film_layers_plus_sigmoid_attention_and_custom_classifier
   phrase_classifier_hidden_size: None
   transf_d_model: None
   transf_nhead: None
   transf_dim_feedforwa

Checkpoint successfully loaded!
--------------------------------------------------
13) Defining log_metrics_handler ...
MetricsLogger :: we'll be logging to /mnt/data/pamessina/workspaces/medvqa-workspace/models/phrase_grounding/20240719_192640_cxrlt2024(Off)_PhraseGrounder(dn121,FiLM_SigmoidAttention,128,256,256,256,1,256-128)/metrics_logs.csv
--------------------------------------------------
14) Running trainer engine ...
---- Epoch 1/200
(1) Training stage (lr = 0.000001) ...
loss 1.57756, cxrlt2024o_att_reg_loss 0.14510, cxrlt2024o_phrcls_loss 1.43246, cxrlt2024o_prc_auc 0.32229, 100.58 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09373, cxrlt2024o_phrcls_loss 1.10629, cxrlt2024o_prc_auc 0.23890, 36.03 secs
New checkpoint saved: checkpoint_1_cxrlt2024o_prc_auc=0.2389.pt
---- Epoch 2/200
(1) Training stage (lr = 0.000004) ...
loss 1.57656, cxrlt2024o_att_reg_loss 0.14290, cxrlt2024o_phrcls_loss 1.43366, cxrlt2024o_prc_auc 0.31080, 94.57 secs
(2) Validation stage ...
cxrl

cxrlt2024o_att_reg_loss 0.09638, cxrlt2024o_phrcls_loss 1.15878, cxrlt2024o_prc_auc 0.24802, 36.80 secs
New checkpoint saved: checkpoint_24_cxrlt2024o_prc_auc=0.2480.pt
---- Epoch 25/200
(1) Training stage (lr = 0.000007) ...
loss 1.59178, cxrlt2024o_att_reg_loss 0.14001, cxrlt2024o_phrcls_loss 1.45177, cxrlt2024o_prc_auc 0.27632, 100.90 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09714, cxrlt2024o_phrcls_loss 1.13336, cxrlt2024o_prc_auc 0.24612, 36.40 secs
---- Epoch 26/200
(1) Training stage (lr = 0.000003) ...
loss 1.54686, cxrlt2024o_att_reg_loss 0.13692, cxrlt2024o_phrcls_loss 1.40994, cxrlt2024o_prc_auc 0.32173, 103.84 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09565, cxrlt2024o_phrcls_loss 1.14797, cxrlt2024o_prc_auc 0.24619, 36.12 secs
---- Epoch 27/200
(1) Training stage (lr = 0.000002) ...
loss 1.56118, cxrlt2024o_att_reg_loss 0.13892, cxrlt2024o_phrcls_loss 1.42226, cxrlt2024o_prc_auc 0.32491, 103.31 secs
(2) Validation stage ...
cxrlt2024o_att_reg_l

cxrlt2024o_att_reg_loss 0.09499, cxrlt2024o_phrcls_loss 1.15892, cxrlt2024o_prc_auc 0.24495, 36.91 secs
---- Epoch 51/200
(1) Training stage (lr = 0.000002) ...
loss 1.54946, cxrlt2024o_att_reg_loss 0.13666, cxrlt2024o_phrcls_loss 1.41280, cxrlt2024o_prc_auc 0.30587, 102.89 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09606, cxrlt2024o_phrcls_loss 1.15406, cxrlt2024o_prc_auc 0.24618, 37.01 secs
---- Epoch 52/200
(1) Training stage (lr = 0.000001) ...
loss 1.57257, cxrlt2024o_att_reg_loss 0.13933, cxrlt2024o_phrcls_loss 1.43324, cxrlt2024o_prc_auc 0.32320, 103.22 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09572, cxrlt2024o_phrcls_loss 1.16248, cxrlt2024o_prc_auc 0.24644, 35.59 secs
---- Epoch 53/200
(1) Training stage (lr = 0.000080) ...
loss 1.58250, cxrlt2024o_att_reg_loss 0.13764, cxrlt2024o_phrcls_loss 1.44485, cxrlt2024o_prc_auc 0.26835, 103.13 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09572, cxrlt2024o_phrcls_loss 1.16680, cxrlt2024o_prc_auc 0

---- Epoch 77/200
(1) Training stage (lr = 0.000080) ...
loss 1.57666, cxrlt2024o_att_reg_loss 0.13859, cxrlt2024o_phrcls_loss 1.43806, cxrlt2024o_prc_auc 0.28800, 103.31 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09971, cxrlt2024o_phrcls_loss 1.14204, cxrlt2024o_prc_auc 0.23190, 36.72 secs
---- Epoch 78/200
(1) Training stage (lr = 0.000043) ...
loss 1.52499, cxrlt2024o_att_reg_loss 0.13566, cxrlt2024o_phrcls_loss 1.38934, cxrlt2024o_prc_auc 0.34729, 103.32 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09477, cxrlt2024o_phrcls_loss 1.20610, cxrlt2024o_prc_auc 0.23848, 38.10 secs
---- Epoch 79/200
(1) Training stage (lr = 0.000023) ...
loss 1.48863, cxrlt2024o_att_reg_loss 0.13693, cxrlt2024o_phrcls_loss 1.35170, cxrlt2024o_prc_auc 0.40014, 100.64 secs
(2) Validation stage ...
cxrlt2024o_att_reg_loss 0.09136, cxrlt2024o_phrcls_loss 1.18274, cxrlt2024o_prc_auc 0.23845, 37.01 secs
---- Epoch 80/200
(1) Training stage (lr = 0.000012) ...
loss 1.48062, cxrlt2024o_att